# Pre-processing the childmind questionnaire data

This notebook aims to:

1. Drop aggregate variables; and

2. Process the diagnosis columns into binary variables.

### 1. Drop aggregate variables

In [ ]:
import ast 
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('resources/data/unsplit_no_totals.csv')

In [ ]:
df.columns

In [ ]:
for col in df.columns:
    suffix = col.split('_')[-1]
    if col in ['EID', 'Sex', 'Age', 'Dx', 'Anx', 'adhd', 'asd']: 
        pass
    else:
        suffix = col.split('_')[-1]
        if not suffix.isdigit():
            del df[col]

In [ ]:
df.to_csv('resources/data/drop_aggregates.csv',index=False)

### 2. Process the diagnosis columns into binary variables

In [ ]:
all_diagnoses = []
for diags in df['Dx'].values.flatten():
    all_diagnoses.extend(ast.literal_eval(diags))

In [ ]:
n = len(df)

In [ ]:
for diagnosis in np.unique(all_diagnoses):
    print diagnosis
    df[diagnosis] = df['Dx'].copy()
    def diag_to_binary(entry):
        return diagnosis in entry
    df[diagnosis] = df[diagnosis].apply(diag_to_binary)
del df['Dx']
del df['Anx']
del df['adhd']
del df['asd']

In [ ]:
df.to_csv('resources/data/init_data.csv',index=False)